<a href="https://colab.research.google.com/github/prateekguptaiiitk/Language-Generation/blob/master/LangGen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from os import listdir
from keras.layers import Layer
import keras.backend as K
from keras.models import Model,load_model
from keras.optimizers import RMSprop
from keras import layers
import numpy as np
from collections import defaultdict
import os, string
import random
from random import shuffle
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.layers.embeddings import Embedding
from keras.layers import Input, Dense, LSTM

In [0]:
#@@ text file prep code @@

with open('Flickr8k.token.txt', 'r') as f:
  data = f.read()
  
data = data.split('\n')
minLenData = list()
for i in range(0,len(data),4):
  minLenData.append(min(data[i:i+4], key = len))

corpora = list()
for desc in minLenData:
  if len(desc.split()) > 0:
    if desc.split()[-1] == '.':
      desc = desc.split()[1:-1]
    else:
      desc = desc.split()[1:]
    corpora.append('sos '+' '.join(desc)+' eos')
corpora = '\n'.join(corpora)

with open('Mincorpus.txt', 'w') as f:
  f.write(corpora)

In [0]:
import nltk
nltk.download('all')

with open('Mincorpus.txt', 'r') as f:
        data = f.read()
data = data.split('\n')

trainData = []
helpingVerb = ['is','am','are','was','were','will','would','shall', 'should']

for sentence in data:
  word = nltk.word_tokenize(sentence)
  tagged = nltk.pos_tag(word[1:-1])
  keys = list()
  count = 0
  for word, tag in tagged:
    if tag[0] == 'N' and count<5:
      keys.append(word)
      count+=1

    if tag[0] == 'V' and word not in helpingVerb and count<5:
      keys.append(word)
      count+=1

    if tag[0] == 'J' and count<5:
      keys.append(word)
      count+=1

    if tag[0] == 'R' and count<5:
      keys.append(word)
      count+=1

  trainData.append(','.join(keys)+ ': '+ sentence)
  
with open('Train.txt', 'w') as fp:
  fp.write('\n'.join(trainData))

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Package cess_cat is already up-to-date!
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Package cess_esp is already up-to-date!
[nltk_data]    | Downloading packag

In [0]:
def ReadandClean(filename):
  
  with open(filename, 'r') as f:
    data = f.read()
  cleanedCorpus = dict()  
  data = data.split('\n')
  print(len(data))
  table = str.maketrans('', '', string.punctuation)
  stopwords = ['a','an','the']
  for line in data:
    key, desc = line.split(':')
    key = key.split(',')
    key = [word.lower() for word in key]
    key = [w.translate(table) for w in key]
    key = [word for word in key if word.isalpha()]
    key = [word for word in key if word not in stopwords]
    desc = desc.strip()
    desc = desc.split(' ')
    desc = [word.lower() for word in desc]
    desc = [w.translate(table) for w in desc]
    desc = [word for word in desc if word.isalpha()]
    desc = [word for word in desc if word not in stopwords]
    cleanedCorpus[' '.join(key)] = desc
    
  return cleanedCorpus
    
def vocabulary(corpus):
  
  vocab = list()
  for key in corpus:
    desc = corpus[key]
    vocab = vocab + desc
  return set(vocab)

def All_sentences(corpus):
  sentences = list()
  for key in corpus:
    sentences.append(corpus[key])
  return sentences
    
filename = 'Train.txt'
corpus = ReadandClean(filename)
vocab = sorted(list(vocabulary(corpus)))
sentences = All_sentences(corpus)
vocab_size = len(vocab)
maxLen = len(max(sentences, key = len))
token_index = dict([(word, i+1) for i, word in enumerate(vocab)])
reverse_token_index = dict((i, word) for word, i in token_index.items())
print(len(corpus),len(vocab))

10115
9636 3975


In [0]:
def oneHot(line,token_index,mode):
  idxlist = []
  if mode == 'key':
    for word in line.split():
      idxlist.append(token_index[word])
  else:
    for word in line.split()[:-1]:
      idxlist.append(token_index[word])
    
  return idxlist

def train_data(token_index):  
  maxLen = len(max(sentences, key = len))
  encoded_keys = [oneHot(key, token_index,'key') for key in corpus]
  padded_keys = pad_sequences(encoded_keys, maxlen=5, padding='post')
  encoded_desc = [oneHot(' '.join(corpus[key]), token_index,'desc') for key in corpus]
  padded_desc = pad_sequences(encoded_desc, maxlen=maxLen, padding='post')
  encoder_x = np.array(padded_keys)
  decoder_x = np.array(padded_desc)
  decoder_y = np.zeros((len(corpus),maxLen,vocab_size+1))
  r,c,_ = decoder_y.shape
  for i in range(r):
    for j in range(1,c):
      if decoder_x[i][j] != 0:
        decoder_y[i][j-1][decoder_x[i][j]] = 1

  return encoder_x, decoder_x, decoder_y

In [0]:
model_1 = load_model('LanGen (2).h5')
weights2set = dict()
for layer in model_1.layers:
  print(layer.name)
  weights2set[layer.name] = layer.get_weights()
  
#weights2set




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where










input_1
input_2
embedding_1
embedding_2
lstm_1
lstm_2
dense_1


In [0]:
encoder_inputs = Input(shape=(None,))

enc_emb_layer = Embedding(vocab_size+1, 50, mask_zero = True, weights=weights2set['embedding_1'])
x = enc_emb_layer(encoder_inputs)
enc_lstm_layer = LSTM(50,return_state=True, weights=weights2set['lstm_1'])
x, state_h, state_c = enc_lstm_layer(x)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None,))

dec_emb_layer = Embedding(vocab_size+1, 50, mask_zero = True, weights=weights2set['embedding_2'])
x = dec_emb_layer(decoder_inputs)
dec_lstm_layer = LSTM(50, return_sequences=True, return_state= True, weights=weights2set['lstm_2'])
x, _, _ = dec_lstm_layer(x, initial_state=encoder_states)
#att_out = attention()(x)
dec_dense =  Dense(vocab_size+1, activation='softmax', weights=weights2set['dense_1'])
decoder_outputs = dec_dense(x)
model_2 = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model_2.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, None, 50)     198800      input_5[0][0]                    
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, None, 50)     198800      input_6[0][0]                    
____________________________________________________________________________________________

In [0]:
encoder_x, decoder_x, decoder_y = train_data(token_index)
optimizer = RMSprop(lr=0.01)
model_2.compile(optimizer='rmsprop', loss='categorical_crossentropy',  metrics = ['acc'])

In [0]:
model_2.fit([encoder_x, decoder_x], decoder_y,
          batch_size=24,
          epochs=10,
          initial_epoch = 0,
          validation_split=0.25, shuffle = True)

In [0]:
#model_2.save('LanGen.h5')

In [0]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(50,))
decoder_state_input_c = Input(shape=(50,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs)
decoder_outputs2, state_h2, state_c2 = dec_lstm_layer(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]

decoder_outputs2 = dec_dense(decoder_outputs2)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [0]:
def decode_sequence(input_seq, token_index, reverse_token_index):
    states_value = encoder_model.predict(input_seq)    
    target_seq = np.zeros((1,1))
    
    target_seq[0, 0] = token_index['sos']
    
    stop_condition = False
    decoded_sentence = ''
    
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        #print(sampled_token_index)
        sampled_char = reverse_token_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char
        #print(sampled_char)
        if (sampled_char == 'eos' or len(decoded_sentence.split()) >= 10):
            stop_condition = True
        
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index
        
        states_value = [h, c]
    
    return decoded_sentence

In [0]:
inp_list = ['dog running beach', 'basketball game', 'woman yellow jogging', 'couple pose wedding day']
for inp_str in inp_list:
  inp_seq = [oneHot(inp_str, token_index, 'key')]
  out_seq = decode_sequence(inp_seq,token_index, reverse_token_index)
  print(' inp_seq : ',inp_str,'\n', 'out_seq :', out_seq)

 inp_seq :  dog running beach 
 out_seq :  dog is in park with her head and white toy
 inp_seq :  basketball game 
 out_seq :  basketball at game is in their stick with ice in
 inp_seq :  woman yellow jogging 
 out_seq :  woman is in park with two holding stick outside and
 inp_seq :  couple pose wedding day 
 out_seq :  couple on of building are each other leaves for snow


In [0]:
import nltk
from nltk.corpus import stopwords

stops = set(stopwords.words('english'))


for w in vocab:
    if w.lower() in stops:
        print (w)